In [ ]:
!pip install -q bitsandbytes datasets accelerate loralib
!pip install -q git+https://github.com/huggingface/peft.git git+https://github.com/huggingface/transformers.git
!pip install -q scipy

In [ ]:
import os
import torch
import torch.nn as nn
import bitsandbytes as bnb
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM

In [ ]:
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "ivr_model_llma_final"

peft_model_id = "/kaggle/input/ivr-final-model-bak/ivr_model_llma_final"
config = PeftConfig.from_pretrained(peft_model_id)
model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path, return_dict=True, load_in_8bit=True, device_map='auto')
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

# Load the Lora model
model = PeftModel.from_pretrained(model, peft_model_id)

In [ ]:
from IPython.display import display, Markdown

def make_inference(hedis_measure):

    batch = tokenizer(f"### Please generate three questions for the customer having the following \n\n### Hedis Measure:\n{hedis_measure}\n", return_tensors='pt')
    batch = batch.to(torch.device('cuda'))

    with torch.cuda.amp.autocast():
      output_tokens = model.generate(**batch, max_new_tokens=250)
    # print(tokenizer.decode(output_tokens[0]))
    display(Markdown((tokenizer.decode(output_tokens[0], skip_special_tokens=True))))

In [ ]:
torch.cuda.empty_cache()
# hedis_measure = "Cardiac Rehabiliation"
hedis_measure = "Lead screening in children"
make_inference(hedis_measure)